In [6]:
from rdflib import Graph, Namespace

# Создаем граф и загружаем существующую онтологию
g = Graph()
g.parse("../dota2_ontology_v2.rdf", format="xml")  # Замените путь на путь к вашему RDF файлу

# Определяем пространство имен
NS = Namespace("http://www.semanticweb.org/root/ontologies/2025/0/dota2#")
g.bind("dota2", NS)

# 1. Герои, которые хорошо показывают себя в игре против своих родственников

In [8]:
query_1 = """
PREFIX dota2: <http://www.semanticweb.org/root/ontologies/2025/0/dota2#>
SELECT ?hero ?relative
WHERE {
  ?hero dota2:bestVersus ?relative .
  ?hero dota2:relativeTo ?relative .
}
"""
for row in g.query(query_1):
    print(f"Герой: {row.hero}, хорош против родственника: {row.relative}")


Герой: http://www.semanticweb.org/root/ontologies/2025/0/dota2#Dragon-Knight, хорош против родственника: http://www.semanticweb.org/root/ontologies/2025/0/dota2#Jakiro
Герой: http://www.semanticweb.org/root/ontologies/2025/0/dota2#Storm-Spirit, хорош против родственника: http://www.semanticweb.org/root/ontologies/2025/0/dota2#Void-Spirit
Герой: http://www.semanticweb.org/root/ontologies/2025/0/dota2#Zeus, хорош против родственника: http://www.semanticweb.org/root/ontologies/2025/0/dota2#Mars


предатели! семья - это главное

# 2. Найти героев, которые являются родственниками, имеют разный тип атаки, и покупают один и тот же предмет

In [11]:
query_2 = """
PREFIX dota2: <http://www.semanticweb.org/root/ontologies/2025/0/dota2#>
SELECT ?hero1 ?hero2 ?item
WHERE {
  ?hero1 dota2:relativeTo ?hero2 .
  ?hero1 dota2:buysItem ?item .
  ?hero2 dota2:buysItem ?item .
  ?hero1 dota2:hero_attack_type ?attackType1 .
  ?hero2 dota2:hero_attack_type ?attackType2 .
  FILTER (?hero1 != ?hero2 && ?attackType1 != ?attackType2)
}
"""
for row in g.query(query_2):
    print(f"Герой 1: {row.hero1}, Герой 2: {row.hero2}, Общий предмет: {row.item}")


Герой 1: http://www.semanticweb.org/root/ontologies/2025/0/dota2#Storm-Spirit, Герой 2: http://www.semanticweb.org/root/ontologies/2025/0/dota2#Ember-Spirit, Общий предмет: http://www.semanticweb.org/root/ontologies/2025/0/dota2#oblivion_staff
Герой 1: http://www.semanticweb.org/root/ontologies/2025/0/dota2#Void-Spirit, Герой 2: http://www.semanticweb.org/root/ontologies/2025/0/dota2#Storm-Spirit, Общий предмет: http://www.semanticweb.org/root/ontologies/2025/0/dota2#ogre_axe
Герой 1: http://www.semanticweb.org/root/ontologies/2025/0/dota2#Ember-Spirit, Герой 2: http://www.semanticweb.org/root/ontologies/2025/0/dota2#Storm-Spirit, Общий предмет: http://www.semanticweb.org/root/ontologies/2025/0/dota2#oblivion_staff
Герой 1: http://www.semanticweb.org/root/ontologies/2025/0/dota2#Earth-Spirit, Герой 2: http://www.semanticweb.org/root/ontologies/2025/0/dota2#Storm-Spirit, Общий предмет: http://www.semanticweb.org/root/ontologies/2025/0/dota2#ogre_axe
Герой 1: http://www.semanticweb.org/r

# 3. Найти героев, у которых меньше всего ролей

In [17]:
query_3 = """
PREFIX dota2: <http://www.semanticweb.org/root/ontologies/2025/0/dota2#>
SELECT ?hero
WHERE {
  ?hero dota2:hasRole ?role .
}
GROUP BY ?hero
HAVING (COUNT(?role) < 3)
"""
for row in g.query(query_3):
    print(f"Герой с 1 или 2 ролями: {row.hero}")


Герой с 1 или 2 ролями: http://www.semanticweb.org/root/ontologies/2025/0/dota2#Phantom-Assassin
Герой с 1 или 2 ролями: http://www.semanticweb.org/root/ontologies/2025/0/dota2#Weaver
Герой с 1 или 2 ролями: http://www.semanticweb.org/root/ontologies/2025/0/dota2#Dawnbreaker
Герой с 1 или 2 ролями: http://www.semanticweb.org/root/ontologies/2025/0/dota2#Shadow-Fiend
Герой с 1 или 2 ролями: http://www.semanticweb.org/root/ontologies/2025/0/dota2#Zeus
Герой с 1 или 2 ролями: http://www.semanticweb.org/root/ontologies/2025/0/dota2#Sniper
Герой с 1 или 2 ролями: http://www.semanticweb.org/root/ontologies/2025/0/dota2#Templar-Assassin
Герой с 1 или 2 ролями: http://www.semanticweb.org/root/ontologies/2025/0/dota2#Pugna
Герой с 1 или 2 ролями: http://www.semanticweb.org/root/ontologies/2025/0/dota2#Chen
Герой с 1 или 2 ролями: http://www.semanticweb.org/root/ontologies/2025/0/dota2#Lich
Герой с 1 или 2 ролями: http://www.semanticweb.org/root/ontologies/2025/0/dota2#Techies
Герой с 1 или 2 ро

# 4. 3 пары героев, которые хороши против одних и тех же героев

In [29]:
query_most_common_best_versus = """
PREFIX dota2: <http://www.semanticweb.org/root/ontologies/2025/0/dota2#>
SELECT ?hero1 ?hero2 (COUNT(?commonEnemy) AS ?commonCount)
WHERE {
  ?hero1 dota2:bestVersus ?commonEnemy .
  ?hero2 dota2:bestVersus ?commonEnemy .
  FILTER (?hero1 != ?hero2)
}
GROUP BY ?hero1 ?hero2
ORDER BY DESC(?commonCount)
LIMIT 3
"""
for row in g.query(query_most_common_best_versus):
    print(f"Герой 1: {row.hero1}, Герой 2: {row.hero2}, Общее количество bestVersus: {row.commonCount}")


Герой 1: http://www.semanticweb.org/root/ontologies/2025/0/dota2#Alchemist, Герой 2: http://www.semanticweb.org/root/ontologies/2025/0/dota2#Arc-Warden, Общее количество bestVersus: 6
Герой 1: http://www.semanticweb.org/root/ontologies/2025/0/dota2#Zeus, Герой 2: http://www.semanticweb.org/root/ontologies/2025/0/dota2#Centaur-Warrunner, Общее количество bestVersus: 6
Герой 1: http://www.semanticweb.org/root/ontologies/2025/0/dota2#Centaur-Warrunner, Герой 2: http://www.semanticweb.org/root/ontologies/2025/0/dota2#Zeus, Общее количество bestVersus: 6


# 5. Соперники выбрали своих героев. Нам нужно найти наиболее подходящих, чтобы выбрать и победить

In [27]:
query_good_against_pugna_and_crystal_maiden = """
PREFIX dota2: <http://www.semanticweb.org/root/ontologies/2025/0/dota2#>
SELECT ?hero
WHERE {
  ?hero dota2:bestVersus dota2:Dazzle .
  ?hero dota2:bestVersus dota2:Snapfire .
}
"""
for row in g.query(query_good_against_pugna_and_crystal_maiden):
    print(f"Герой, который хорош против Dazzle и Snapfire: {row.hero}")


Герой, который хорош против Dazzle и Snapfire: http://www.semanticweb.org/root/ontologies/2025/0/dota2#Juggernaut
Герой, который хорош против Dazzle и Snapfire: http://www.semanticweb.org/root/ontologies/2025/0/dota2#Luna
